# Jupyter-Notebook für ein eigenes KNN

Hier wird ein KNN trainiert und getestet zum Erkennen von Ziffern.

Die Ziffern (0, 1, ..., 9) liegen als Graustufenbilder in der Auflösung 28x28 vor, sind als CSV-Datei verfügbar als sog. MNIST-Datensatz.

- **Code für ein 3-schichtiges KNN (Input - Hidden - Output)**
- **Code zum Trainieren des Netzes für die MNIST-Daten**

## Import der notwendigen Bibliotheken

In [ ]:
import numpy as np
from scipy.special import expit
import matplotlib.pyplot as plt
import pandas as pd
import imageio.v2 as imageio
import glob

# ensure the plots are inside this notebook, not an external window
%matplotlib inline

## Python-Klasse für ein neuronales Netz

**3-schichtiges Neuronales Netz**

Parameter:
- Anzahl Neuronen des Input Layer
- Anzahl Neuronen des Hidden Layer
- Anzahl Neuronen des Output-Layer
- Lernrate

In [ ]:
# neural network class definition
class neuralNetwork:
    
    
    # initialise the neural network
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        self.wih = np.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))

        # learning rate
        self.lr = learningrate
        
        # activation function is the sigmoid function
        self.activation_function = lambda x: expit(x)
        
        pass

    
    # train the neural network
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = np.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = np.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = np.dot(self.who.T, output_errors) 
        
        # update the weights for the links between the hidden and output layers
        self.who += self.lr * np.dot((output_errors * final_outputs * (1.0 - final_outputs)), np.transpose(hidden_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.lr * np.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), np.transpose(inputs))
        
        pass

    
    # query the neural network
    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = np.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = np.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = np.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

## Parameter setzen und KNN aufbauen

In [ ]:
# number of input, hidden and output nodes
input_nodes = 784
hidden_nodes = 200
output_nodes = 10

# learning rate
learning_rate = 0.1

# create instance of neural network
knn = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

## Trainingsdaten (ggf. entpacken und) laden

Falls die CSV-Datei nur gezipped vorliegt:
- führen Sie die folgende Zelle aus, und dann nicht(!) die übernächste

ansonsten
- führen Sie nur die übernächste Zelle aus.

In [ ]:
import zippen

training_data_list = zippen.zip_entpacken ("mnist_train", "mnist_dataset")

In [ ]:
# load the mnist training data CSV file into a list
training_data_file = open("mnist_dataset/mnist_train.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

- Der Trainingsdatensatz ist zeilenweise aufgebaut. 
- Jede Zeile ist jetzt als String in einer Liste vorhanden.
- Dieser String enthält 785 Integerwerte, jeweils durch ein Komma getrennt.
  - Der erste Eintrag ist die dargestellt Ziffer
  - die folgenden 784 Einträge sind die 28x28 Pixelwerte, also Werte aus [0, 255]
  
Beispielsweise hier der 0-te Datensatz:


In [ ]:
training_data_list[0]

## KNN trainieren

In [ ]:
# train the neural network

# epochs is the number of times the training data set is used for training
epochs = 10
print ("Bitte warten!")

for e in range(epochs):
    print(e+1, "von", epochs, "Epochen gestartet.... ", end = "")
    # Alle Einträge in den Trainingsdaten werden durchgegangen
    for record in training_data_list:
        # Aufspalten der Dateneinträge (Komma-separiert)
        all_values = record.split(',')
        # Skalieren der Werte in den Bereich zwischen 0 und 1
        inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = np.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        knn.train(inputs, targets)
        pass
    print ("done")
    pass
print ("Done")

## Alle Trainingsdaten testen

***Zur Kontrolle werden alle Trainingsdaten getestet, da (natürlich) ein KNN in der Regel nicht notwendigerweise alle Trainingsdaten korrekt klassifiziert.***

In [ ]:
def test_all_data(data_list):
    korrekte = 0
    anzahl = len(data_list)
    wrongIndizes = []
    
    for index in range (anzahl):
        
        img_array = np.asfarray(data_list[index].split(","))[1:].astype(int)
        img_data  = img_array
        img_data = (img_data / 255.0 * 0.99) + 0.01

        # data is remaining values
        inputs = img_data

        # query the network
        outputs = knn.query(inputs)
        
        # the index of the highest value corresponds to the label
        label = np.argmax(outputs)
        correct_label = int(data_list[index][0])
 
        if (label == correct_label):
            korrekte += 1
        else:
            wrongIndizes.append(index)
    print ("Von den", anzahl, "Daten sind", korrekte, "korrekt,", anzahl - korrekte, "falsch zugeordnet")
    # print(wrongIndizes)


In [ ]:
test_all_data(training_data_list)

## Wie gut wurden die Trainingsdaten klassifiziert?

Zunächst eine Hilfsfunktion:
- `data_list` enthält wahlweise die Trainings- oder die Testdaten
- `index` ist der Index des untersuchten Eintrags in der Liste

Die Funktion liefert ein Tupel von Labeln:
- Dasjenige Label, das der Eintrag laut Liste haben soll
- Das Label das das KNN liefert.

In [ ]:
def soll_ist (index, data_list):
    img_array = np.asfarray(data_list[index].split(","))[1:].astype(int)
    img_data  = img_array
    img_data = (img_data / 255.0 * 0.99) + 0.01
    inputs = img_data

    # query the network
    outputs = knn.query(inputs)
    label = np.argmax(outputs)
    correct_label = int(data_list[index][0])
    return (correct_label, label)

Mit der folgenden Funtion kann man alle Daten testen. Es wird eine Numpy-Matrix erzeugt.

In [ ]:
def soll_ist_all(data_list):
    m = np.zeros((10, 10), dtype = int)
    anzahl = len(data_list)
    for index in range (anzahl):
        s, i = soll_ist (index, data_list)
        m [s][i] += 1
    return m

In der folgenden Tabelle sind die Fehler aufgelistet.

- Steht z.B. in der 3-ten Zeile der Wert 17 in der 4-ten Spalte, wurde 17 mal die Ziffer `3` fälschlicherweise als Ziffer `4` erkannt.

In [ ]:
sia_train = soll_ist_all(training_data_list)
df = pd.DataFrame(data=sia_train)
df["Fehler"] = ""

for i in range (10):
    f = 0;
    for j in range (10):
        if i != j:
            inhalt = int (df[j][i])
            f += int (inhalt)
    df["Fehler"][i] = f

display (df)

## Wie gut werden mit diesem KNN die Testdaten klassifiziert?

In [ ]:
# load the mnist training data CSV file into a list
test_data_file = open("mnist_dataset/mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

Hier sieht man den 0-ten Datensatz der Testdaten

In [ ]:
test_data_list[0]

In [ ]:
sia_test = soll_ist_all(test_data_list)

df = pd.DataFrame(data=sia_test)
df["Fehler"] = ""

for i in range (10):
    f = 0;
    for j in range (10):
        if i != j:
            inhalt = int (df[j][i])
            f += int (inhalt)
    df["Fehler"][i] = f

display (df)

## Eigene Bilder laden

**Bitte die geünschten Testdaten angeben!**

- Man kann mehrere Bilder testen. Alle Bilder liegen in einem Unterverzeichnis.
- Der Name jeder Bild-Datei in diesem Unterverzeichnis hat die Form "xxx_n.png", wobei "xxx" ein beliebiger Text ist und "n" das Label (also die in dem Bild dargestellt Ziffer) ist

In [ ]:
# our own image test data set
our_own_dataset = []

# load the png image data as test data set
for image_file_name in glob.glob('my_own_images/2828_my_own_?.png'):
#for image_file_name in glob.glob('Meine_Fotos/2828_bov_?.png'):
    
    # use the filename to set the correct label
    label = int(image_file_name[-5:-4])
    
    # load image data from png files into an array
    print ("loading ... ", image_file_name)
    img_array = imageio.imread(image_file_name, as_gray=True)
    
    # reshape from 28x28 to list of 784 values, invert values
    img_data  = 255.0 - img_array.reshape(784)
    
    # then scale data to range from 0.01 to 1.0
    img_data = (img_data / 255.0 * 0.99) + 0.01
    
    # append label and image data  to test data set
    record = np.append(label,img_data)
    our_own_dataset.append(record)
    
    pass

## Eine eigenes Bild aus dieser Bildersammlung testen

In [ ]:
# Nummer des Testbildes:
# record to test
item = 4

In [ ]:
# test the neural network with our own images

# plot image
plt.imshow(our_own_dataset[item][1:].reshape(28,28), cmap='Greys', interpolation='None')

# correct answer is first value
correct_label = our_own_dataset[item][0]
# data is remaining values
inputs = our_own_dataset[item][1:]

# query the network
outputs = knn.query(inputs)
for i in range (output_nodes):
    print (f'Güte der Erkennung = {outputs[i][0]:1.5f} für die Ziffer {i}')
        

# the index of the highest value corresponds to the label
label = np.argmax(outputs)
print("network says ", label)
# append correct or incorrect to list
if (label == correct_label):
    print ("match!")
else:
    print ("no match!, should be", int(correct_label))
    pass


## Alle eigenen Bilder testen

In [ ]:
for item in range (len(our_own_dataset)):
    
    # correct answer is first value
    correct_label = our_own_dataset[item][0]
    # data is remaining values
    inputs = our_own_dataset[item][1:]

    # query the network
    outputs = knn.query(inputs)

    # the index of the highest value corresponds to the label
    label = np.argmax(outputs)
    print("Bildnummer", item, ": network says:", label, "; shoud be:", int(correct_label))
    # append correct or incorrect to list
    if (label == correct_label):
        print ("match!")
    else:
        print ("no match!")

## Ein eigenes Bild testen

In [ ]:
def testen (image_file_name, correct_label):
    img_array = imageio.imread(image_file_name, as_gray=True)
    
    # reshape from 28x28 to list of 784 values, invert values
    img_data  = 255.0 - img_array.reshape(784)
    
    # then scale data to range from 0.01 to 1.0
    img_data = (img_data / 255.0 * 0.99) + 0.01
    #print(numpy.min(img_data))
    #print(numpy.max(img_data))
        
    # plot image
    plt.imshow(img_data.reshape(28,28), cmap='Greys', interpolation='None')

    # data is remaining values
    inputs = img_data

    # query the network
    outputs = knn.query(inputs)
    for i in range (output_nodes):
        print (f'Güte der Erkennung {outputs[i][0]:1.5f} für die Ziffer {i}')
        
    
    # the index of the highest value corresponds to the label
    label = np.argmax(outputs)
    print("network says ", label)

    # append correct or incorrect to list
    if (label == correct_label):
        print ("match!")
    else:
        print ("no match!, should be", int(correct_label))

    
testen ('Meine_Fotos/2828_bov_4.png', 5)
    
 